In [1]:
import pandas as pd
import numpy as np
pd.set_option('expand_frame_repr', False)

#### load csvs to pandas dataframes, we will use online data for the best accuracy

In [2]:
search_trend_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/search_trends_symptoms_dataset/United%20States%20of%20America/2020_US_weekly_symptoms_dataset.csv"
hospitalization_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/cc_by/aggregated_cc_by.csv"
search_trend_df = pd.read_csv(search_trend_url)
hospitalization_df = pd.read_csv(hospitalization_url)


### We will try to clean the search trend dataset first

after loading the dataset to pandas dataframe, we will check how many parameters does the dataframe contains

In [3]:
search_trend_df.iloc[np.r_[0:10,-10:0]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Columns: 124 entries, open_covid_region_code to symptom:Yawn
dtypes: float64(121), object(3)
memory usage: 558.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 433 entries, 0 to 432
Columns: 125 entries, open_covid_region_code to hospitalized_new
dtypes: datetime64[ns](1), float64(122), object(2)
memory usage: 426.2+ KB
None
                                      sum
symptom:Aphonia                   4478.88
symptom:Viral pneumonia           4136.85
symptom:Shallow breathing         3902.18
symptom:Angular cheilitis         3526.01
symptom:Dysautonomia              3435.65
symptom:Laryngitis                3280.74
symptom:Crackles                  3268.78
symptom:Ventricular fibrillation  3209.33
symptom:Rectal pain               3170.71
symptom:Hemolysis                 3168.77


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Since we have several NaN columns, we can drop the columns where all data in the column are NaN

And also, by looking at the dataset, columns named as `country_region_code`, `country_region`, and `sub_region_1_code` can be dropped.

search_trend_df = search_trend_df.dropna(axis=1, how='all')
search_trend_df = search_trend_df.drop(['country_region_code', 'country_region', 'sub_region_1_code'], axis=1)
search_trend_df = search_trend_df.rename({'sub_region_1': 'region_name'}, axis=1)
# now, check the hospitalization cases

print(search_trend_df.info())

hospitalization_df = pd.read_csv('aggregated_cc_by.csv')

# since we have several NaN columns, we can drop the columns with null values to get better result
hospitalization_df = hospitalization_df[['open_covid_region_code', 'region_name', 'date', 'hospitalized_new']]
hospitalization_df = hospitalization_df.dropna(axis=0, how='all')

Yet, we still see a lot of NaNs, let's check the number of NaNs in each column

In [7]:
search_trend_df.isnull().sum()

open_covid_region_code                0
sub_region_1                          0
date                                  0
symptom:Adrenal crisis              346
symptom:Ageusia                     370
                                   ... 
symptom:Urinary urgency             218
symptom:Vasculitis                  535
symptom:Ventricular fibrillation    189
symptom:Viral pneumonia             218
symptom:Yawn                        533
Length: 124, dtype: int64

In case we need to drop several columns that contains too many NaNs later, we will modify code below to achieve that

In [8]:
# cols_drop = []
# for col in search_trend_df.columns.values:
#     if search_trend_df[col].isnull().sum()/ (float)len(search_trend_df) > .75
#         cols_drop.append(col)
# search_trend_df = search_trend_df.drop(cols_drop, axis=1)

### Now, it's time to clean the hospitalization dataset

We still need to have a look at the dataset first

In [9]:
hospitalization_df

,open_covid_region_code,region_name,date,cases_cumulative,cases_new,cases_cumulative_per_million,cases_new_per_million,deaths_cumulative,deaths_new,deaths_cumulative_per_million,...,stringency_index,stringency_index_for_display,stringency_legacy_index,stringency_legacy_index_for_display,government_response_index,government_response_index_for_display,containment_health_index,containment_health_index_for_display,economic_support_index,economic_support_index_for_display
0,AFG,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
2,AFG,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
3,AFG,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
4,AFG,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98429,VUT,Vanuatu,2020-10-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.22,22.22,28.57,28.57,25.64,25.64,25.76,25.76,25.0,25.0
98430,VUT,Vanuatu,2020-10-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,22.22,22.22,28.57,28.57,25.64,25.64,25.76,25.76,25.0,25.0
98431,VUT,Vanuatu,2020-10-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22.22,NaN,28.57,NaN,25.64,NaN,25.76,NaN,25.0
98432,VUT,Vanuatu,2020-10-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,22.22,NaN,28.57,NaN,25.64,NaN,25.76,NaN,25.0


Since we only need `open_covid_region_code`, `date`, and `hospitalized_new`, we will select only these columns

And in this mini-project only, we will use only data gathered from US


In [10]:
hospitalization_df = hospitalization_df[['open_covid_region_code','date','hospitalized_new']]
hospitalization_df = hospitalization_df[hospitalization_df['open_covid_region_code'].str.contains('^US-')]
hospitalization_df

,open_covid_region_code,date,hospitalized_new
84830,US-WY,2020-03-07,0.0
84831,US-WY,2020-03-08,0.0
84832,US-WY,2020-03-09,0.0
84833,US-WY,2020-03-10,0.0
84834,US-WY,2020-03-11,0.0
...,...,...,...
97019,US-AK,2020-10-03,0.0
97020,US-AK,2020-10-04,0.0
97021,US-AK,2020-10-05,0.0
97022,US-AK,2020-10-06,0.0


Since the data here are daily basis, and the data in search_trend_df are weekly basis, we need to convert these to weekly basis

In [11]:
hospitalization_df['date'] = pd.to_datetime(hospitalization_df['date'], format='%Y-%m-%d')
hospitalization_df = hospitalization_df.groupby(['open_covid_region_code',]).resample('W', on='date',loffset='1d').sum()
hospitalization_df = hospitalization_df.reset_index()
result = search_trend_df.merge(hospitalization_df, on=['open_covid_region_code', 'region_name', 'date'])
result.reset_index()
print(result.info())
result.to_csv("result.csv", index=False)
print(pd.DataFrame(result[result.columns[3:-1]].sum(axis=0), columns=['sum']).sort_values(by=['sum'], ascending=False).head(10))

# print(pd.DataFrame(result.sum(axis=0), columns=['sum']).sort_values(by=['sum']))


,open_covid_region_code,date,hospitalized_new
0,US-AK,2020-03-09,0.0
1,US-AK,2020-03-16,0.0
2,US-AK,2020-03-23,0.0
3,US-AK,2020-03-30,6.0
4,US-AK,2020-04-06,14.0
...,...,...,...
1795,US-WY,2020-09-14,9.0
1796,US-WY,2020-09-21,15.0
1797,US-WY,2020-09-28,19.0
1798,US-WY,2020-10-05,20.0


now the cleanning is done, we merge the datasets

We also drop column `open_covid_region_code` since we already have `sub_region_1` 

For future purpose, we rename `sub_region_1` to `region_name`

In [12]:
search_trend_df['date'] = pd.to_datetime(search_trend_df['date'], format='%Y-%m-%d')
result_df = search_trend_df.merge(hospitalization_df, on=['open_covid_region_code', 'date'])
result_df = result_df.rename({'sub_region_1': 'region_name'}, axis=1)
result_df = result_df.drop('open_covid_region_code',axis=1)
result_df

,region_name,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amblyopia,symptom:Amenorrhea,symptom:Angular cheilitis,symptom:Anosmia,symptom:Aphonia,...,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Vasculitis,symptom:Ventricular fibrillation,symptom:Viral pneumonia,symptom:Yawn,hospitalized_new
0,Alaska,2020-03-09,10.60,NaN,11.61,NaN,12.79,NaN,7.07,7.91,...,11.11,13.97,46.95,10.27,NaN,9.26,NaN,18.85,10.77,0.0
1,Alaska,2020-03-16,11.69,NaN,6.39,NaN,NaN,NaN,15.59,7.95,...,14.49,9.04,78.71,12.00,NaN,11.53,NaN,24.94,14.34,0.0
2,Alaska,2020-03-23,11.15,16.57,NaN,NaN,9.53,NaN,45.17,NaN,...,7.63,8.36,37.54,8.80,NaN,6.75,7.04,13.79,15.69,0.0
3,Alaska,2020-03-30,8.96,8.96,7.11,NaN,8.53,NaN,19.05,NaN,...,9.38,14.79,24.46,9.81,7.96,7.54,7.11,8.82,17.63,6.0
4,Alaska,2020-04-06,10.08,7.35,NaN,NaN,10.80,7.06,6.77,NaN,...,8.64,13.83,16.28,10.08,NaN,11.09,NaN,9.07,14.12,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,Wyoming,2020-08-24,4.42,3.31,3.42,5.19,3.42,NaN,2.65,NaN,...,3.87,2.26,3.98,4.09,NaN,3.64,2.48,NaN,3.87,16.0
461,Wyoming,2020-08-31,5.03,2.79,NaN,5.75,4.19,NaN,NaN,NaN,...,3.07,2.57,2.68,NaN,NaN,5.08,NaN,NaN,3.63,8.0
462,Wyoming,2020-09-07,3.36,3.25,2.20,3.63,NaN,2.25,2.70,NaN,...,2.81,NaN,3.19,3.19,NaN,4.51,NaN,NaN,4.02,4.0
463,Wyoming,2020-09-14,4.15,4.97,NaN,5.14,5.79,NaN,4.59,NaN,...,5.63,3.06,4.59,3.22,2.68,3.88,NaN,NaN,4.37,9.0


We may need to use the dataframe in another file, so we export it as a CSV file

And also, we do not want to have index in our generated CSV file, so we use `index=False`

In [13]:
result_df.to_csv('result.csv', index=False)

### The cleaning process is done